# Data Reshaping with Pandas

In data analysis, reshaping a pandas dataframe is a common activity. Changing a table's format from long to wide or wide to long is also known as transposing, pivoting, or unpivoting.

## Wide vs. Long Format

Wide and long formats are two ways of organizing data. Both formats contain the same information but are structured differently.

### Example:
- **Wide Format:** Each variable is in a separate column.
- **Long Format:** Variables are stacked in a single column, with an additional column indicating the variable name.

![Data Transformation 101](https://media.licdn.com/dms/image/D4D12AQENzwkJOrmSMA/article-cover_image-shrink_720_1280/0/1708642953419?e=1724889600&v=beta&t=knn2GFkWyHcNm51cGdscjephJ4L5k2sE6cd1tjaVV5Q)

*Image source: [Data Transformation 101 using pandas.melt to convert wide format data to long format](https://www.linkedin.com/pulse/data-transformation-101-using-pandasmelt-convert-wide-gabriel-ejiro-dklqf/)*



## When to Use Wide or Long Format?

- **Usability:** Consider how the data will be used. For example, a very wide table with 300 columns (one for each date) might be difficult to analyze. Converting it to long format could make it easier to work with.
- **Readability:** Ensure the data is easy to understand. For example, a long table with 300 different values for a key might be more readable in wide format, with each key as a separate column.

## From Long to Wide and Back Again: Pivoting and Unpivoting in Pandas

### Pivoting Columns
**Pivoting** means turning unique values from one column into separate columns, creating a more spread-out table. It's often used to make data more readable and to summarize it in a compact format. In essence, pivoting transforms a long table into a wide table.

#### Example: Pivoting

Imagine you have a table of sales data.

| Date       | Product | Sales |
|------------|---------|-------|
| 2024-01-01 | A       | 10    |
| 2024-01-01 | B       | 20    |
| 2024-01-02 | A       | 15    |
| 2024-01-02 | B       | 25    |


Pivoting this data on the `Product` column will spread out the sales data for each product into separate columns.

| Date       | A  | B  |
|------------|----|----|
| 2024-01-01 | 10 | 20 |
| 2024-01-02 | 15 | 25 |


### Unpivoting Columns
**Unpivoting** (also known as **melting**) is the reverse of pivoting. It takes columns and transforms them into rows. This is useful for converting a wide table into a long table, which is often easier to analyze and visualize.

#### Example
If you start with the pivoted table.

| Date       | A  | B  |
|------------|----|----|
| 2024-01-01 | 10 | 20 |
| 2024-01-02 | 15 | 25 |

Unpivoting it will convert the `A` and `B` columns back into rows:

| Date       | Product | Sales |
|------------|---------|-------|
| 2024-01-01 | A       | 10    |
| 2024-01-01 | B       | 20    |
| 2024-01-02 | A       | 15    |
| 2024-01-02 | B       | 25    |



#### Example 1: Wide to Long Format with `melt()`
Recall that the original Gapminder dataframe is in wide format. This example shows how to convert the Gapminder dataset from wide to long format to stack the `lifeExp`, `pop`, and `gdpPercap` columns into a single column.


```{admonition} **Syntax for Data Reshaping in Pandas:**

`melt()`: Convert from wide to long format

df.melt(id_vars='...', value_vars=['...', '...'], var_name='...', value_name='...')

```

```{admonition} **Description:**
:class: hint
- `df`: The dataframe you want to reshape.

- `id_vars`: Columns to keep fixed (identifiers), which will remain as they are. These columns are not unpivoted.

- `value_vars`: Columns to unpivot. These columns are transformed from wide format to long format.

- `var_name`: Name of the new column that will contain the unpivoted column names. This column will contain the names of the original `value_vars`.

- `value_name`: Name of the new column that will contain the values from the unpivoted columns. This column will contain the values from the original `value_vars`.


```

In [1]:
import pandas as pd

# URL to the raw CSV file on GitHub
url = 'https://raw.githubusercontent.com/kirenz/datasets/master/gapminder.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(url)

In [2]:
# Recall that the original Gapminder dataframe is in wide format

df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [3]:
# Wide to long format
df_long = pd.melt(df, id_vars=['country', 'continent', 'year'], 
                  value_vars=['lifeExp', 'pop', 'gdpPercap'],
                  var_name='variable', value_name='value')
print(df_long.head(10))

       country continent  year variable   value
0  Afghanistan      Asia  1952  lifeExp  28.801
1  Afghanistan      Asia  1957  lifeExp  30.332
2  Afghanistan      Asia  1962  lifeExp  31.997
3  Afghanistan      Asia  1967  lifeExp  34.020
4  Afghanistan      Asia  1972  lifeExp  36.088
5  Afghanistan      Asia  1977  lifeExp  38.438
6  Afghanistan      Asia  1982  lifeExp  39.854
7  Afghanistan      Asia  1987  lifeExp  40.822
8  Afghanistan      Asia  1992  lifeExp  41.674
9  Afghanistan      Asia  1997  lifeExp  41.763


#### Example 2: Long to Wide Format with `pivot()`
This example shows how to converting the reshaped long format of the Gapminder dataset back to its original wide format.


```{admonition} **Syntax for Data Reshaping in Pandas:**

`pivot()`: Convert from long to wide format

df.pivot(index='...', columns='...', values='...')

```

```{admonition} **Description:**
:class: hint
- `df`: The dataframe you want to reshape.

- `index`: Column to use to make new frame's index. This column remains fixed and the values become row indices in the pivoted dataframe.

- `columns`: Column to use to make new frame's columns. The unique values in this column become the new column headers in the pivoted dataframe.

- `values`: Column(s) to use for populating the new frame's values. The data from this column fills the cells in the pivoted dataframe.



```

In [4]:
# Assuming df_long from the previous example
df_wide = df_long.pivot(index=['country', 'continent', 'year'], 
                        columns='variable', values='value').reset_index()
print(df_wide.head())

variable      country continent  year   gdpPercap  lifeExp         pop
0         Afghanistan      Asia  1952  779.445314   28.801   8425333.0
1         Afghanistan      Asia  1957  820.853030   30.332   9240934.0
2         Afghanistan      Asia  1962  853.100710   31.997  10267083.0
3         Afghanistan      Asia  1967  836.197138   34.020  11537966.0
4         Afghanistan      Asia  1972  739.981106   36.088  13079460.0


```{note}
`.reset_index()`: Convert the index of the pivoted dataframe into columns

After pivoting, the index of the resulting dataframe is composed of the specified `index` columns (`country`, `continent`, `year`). Using `.reset_index()` converts these index columns back into regular columns, making the dataframe easier to work with for further analysis.

```
